In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
    %tensorflow_vesion 2.x
except Exception:
    pass
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

1.14.0
False


In [14]:
import pandas as pd

In [15]:
import jieba

In [7]:
comments = pd.read_csv('../../datasource/movie_comments.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [198]:
def stopword(sentence):
    with open("../../datasource/chinese_stopwords.txt", 'r') as f:
        stopwords = [line.replace('\n', '') for line in f.readlines() if line != '\n']
    return [sen for sen in sentence if sen not in stopwords and sen not in ['\r\n', '\r']]       

In [212]:
data, target = [], []
def reformat():
    for i, comm in enumerate(comments.values):
        try:
            comm_re = ''.join(re.findall(r'[\S]', comm[3]))
            cm = stopword(jieba.lcut(comm_re))
            star = int(comm[4])
            if len(cm) > 3 and star:
#                 print(cm)
                data.append(' '.join(cm))
                target.append(1 if star > 2 else 0)            
        except Exception as e:
            print(e)
            continue
        if i%100000 == 0:
            print(i, comm)

In [213]:
reformat()

0 ['1' 'https://movie.douban.com/subject/26363254/' '战狼2'
 '吴京意淫到了脑残的地步，看了恶心想吐' '1']
invalid literal for int() with base 10: 'star'
100000 ['98655' 'https://movie.douban.com/subject/26088403/' '检察官外传 검사외전'
 '善良地活着吧！\n捍卫自己在意的事物！' '4']
200000 [198655 'https://movie.douban.com/subject/25839662/' '45周年 45 Years'
 '#2016十佳候选#“就怕鬼一样的前女友”。被发现的不仅仅是逝去的爱人，还是这个已经年老的欧洲昨日的（充满可能性却错失了的）六十年代。看上去就是个老年中产阶级情节剧，不过低下政治暗流涌动——没有什么比“胎死腹中”更加说明问题的了。'
 5]


In [214]:
sentences = [sentence.split(' ') for sentence in data]

In [215]:
print(len(target))
print(len(sentences))

185467
185467


In [176]:
import gensim

In [180]:
# sentences = gensim.models.word2vec.LineSentence(source=da)

In [217]:
model = gensim.models.Word2Vec(sentences=sentences, min_count=1)

In [89]:
# model = gensim.models.Word2Vec.load('word2vec.model')

In [218]:
# 取所有词平均作为句子向量
sentence_embedding = []
for i, sentence in enumerate(sentences):
    sent_vec = np.zeros((100, ))
    for sen in sentence:
        try:
            sent_vec += model[sen]
        except Exception as e:
            print(e)
            continue
    if np.sum(sent_vec) != 0.:
        sent_vec = sent_vec / len(sentence)
        sentence_embedding.append(sent_vec)
    else:
        print('delete target: ', i)
        target.pop(i)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [221]:
len(target)

185467

In [222]:
import tensorflow as tf
from tensorflow import keras

In [225]:
from sklearn.model_selection import train_test_split

In [224]:
len(target)

185467

In [322]:
train_x, test_x, train_y, test_y = train_test_split(sentence_embedding_bak, target, train_size=0.75)

In [323]:
len(train_x)

139100

In [324]:
len(test_y)

46367

In [331]:
class_model = keras.Sequential()
class_model.add(keras.layers.Embedding(10000 * 100, 16))
class_model.add(keras.layers.GlobalAveragePooling1D())
class_model.add(keras.layers.Dense(16, activation='relu'))
class_model.add(keras.layers.Dense(1, activation='sigmoid'))
class_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 16)          16000000  
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 16,000,289
Trainable params: 16,000,289
Non-trainable params: 0
_________________________________________________________________


In [332]:
class_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'] )

In [334]:
history = class_model.fit(train_x, 
                    train_y, 
                    epochs=40, 
                    batch_size=512, 
                    validation_data=(test_x, test_y), 
                    verbose=1)

Train on 139100 samples, validate on 46367 samples
Epoch 1/40


InvalidArgumentError: indices[93,87] = -1 is not in [0, 1000000)
	 [[{{node embedding_6/embedding_lookup}}]]

In [248]:
len(train_y)

139100

In [252]:
imdb = keras.datasets.imdb

In [253]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 5s 0us/step


In [254]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))


Training entries: 25000, labels: 25000


In [255]:
print(train_data[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [256]:
len(train_data[0])

218

In [257]:
# 一个映射单词到整数索引的词典
word_index = imdb.get_word_index()

# 保留第一个索引
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1646592/1641221 [==============================] - 3s 2us/step


In [258]:
decode_review(train_data[0])


"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [259]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [315]:
train_data

array([[   1,   14,   22, ...,    0,    0,    0],
       [   1,  194, 1153, ...,    0,    0,    0],
       [   1,   14,   47, ...,    0,    0,    0],
       ...,
       [   1,   11,    6, ...,    0,    0,    0],
       [   1, 1446, 7079, ...,    0,    0,    0],
       [   1,   17,    6, ...,    0,    0,    0]], dtype=int32)

In [318]:
sentence_embedding_bak

(185467, 100)

In [272]:
a = np.array([1, 2])
b = np.array([3, 4])

In [273]:
a

array([1, 2])

In [274]:
b

array([3, 4])

In [275]:
a+ b

array([4, 6])

In [283]:
current = sentence_embedding[0]
for i, sen_vec in enumerate(sentence_embedding[1:]):
    print(sen_vec)
    current = np.append([current], [concatenate((a1,a2,…,an),axis = 0)], axis=0)

[ 0.43731922  0.47090725  0.55100796 -0.02978868 -0.06107491 -0.11797836
  0.14650161  0.38459506 -0.37658579 -0.27941765  0.25469433  0.63527286
  0.73321756 -0.247031    0.12732547 -0.71602823 -0.33435991 -0.09277176
  0.33512335  0.41635372 -0.10170784 -0.13374385 -0.4455155  -0.6408533
 -0.88490279  0.02186828  0.04625729 -0.1382017  -0.55579295 -0.04211303
  0.52176595 -0.4379165   0.54244553  0.22698816 -0.32255633  0.04465351
 -0.50102166 -0.2322214  -0.12260795  0.15119684 -0.19645915  0.01781936
 -0.05851697 -0.36260617  0.06775386  0.0708126  -0.12354369  0.46188651
 -0.19552997 -0.30944649  0.73499347 -0.32273283 -0.04424583 -0.40000982
  0.19987981  0.4077309  -0.40231788  0.42537626 -0.05591301 -0.65034528
 -0.45107633 -0.12387632 -0.1951537   0.51567011  0.26495456 -0.29366585
  0.25507762 -0.94908298  0.22495868 -0.0407216   0.03883398  0.63600621
 -0.36291423 -0.23670373 -0.14332732 -0.58976731 -0.09361702  0.77000931
  0.19485069  0.64123743 -0.20577498  0.31816843  0.

ValueError: all the input arrays must have same number of dimensions

In [319]:
sentence_embedding_bak = np.r_[sentence_embedding]

In [320]:
a = sentence_embedding_bak[: 10000]

In [321]:
a

array([[ 0.27169057,  0.51725687,  1.50152302, ..., -0.87800729,
        -0.9030839 , -0.77796282],
       [ 0.43731922,  0.47090725,  0.55100796, ..., -0.57681501,
        -0.22039125, -0.36312132],
       [ 0.33584877,  0.48892233,  0.58424383, ..., -0.50350991,
        -0.51879819, -0.58462038],
       ...,
       [ 0.21610119,  0.68261992,  0.30252305, ..., -0.08886573,
        -0.53425867, -0.57306479],
       [ 0.29380002,  1.09682322,  1.52849071, ..., -1.4219576 ,
        -0.59510534, -0.63166511],
       [ 0.1114275 ,  0.428187  ,  0.43091669, ..., -0.24359849,
        -0.68508477, -0.46024447]])